In [1]:
import os
import numpy as np
import trimesh
from functools import wraps
from time import time

from torch.utils.cpp_extension import load
GPooling = load(name="grid_pooling", sources=["../cpp_files/grid_pooling.cpp"])

def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        t0 = time()
        result = f(*args, **kw)
        t1 = time()
        print(f"--- Function '{f.__name__}' took {t1-t0:.2f}s to run ---")
        return result
    return wrap

In [2]:
data_dir = '../data/car'
filenames = os.listdir(data_dir)
# filename = filenames[0]

# N = 0
# n_samples = 0   
# for filename in tqdm(filenames):
#     if n_samples >= 1500:
#         break
#     file_path = f'../data/{filename}/models/model_normalized.obj'
#     if not os.path.exists(file_path):
#         continue

#     mesh = trimesh.load(file_path, force='mesh')
#     if not isinstance(mesh, trimesh.Trimesh):
#         continue
    
#     N += mesh.vertices.shape[0]
#     n_samples += 1
#     # mesh.scene().show()

# print(f'Total number of points: {N}')

In [3]:
mesh = trimesh.load(f'{data_dir}/{filenames[0]}/models/model_normalized.obj', force='mesh')
point_cloud = mesh.vertices

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FeatureExtractor(nn.Module):
    def __init__(self, n_features_in=3, n_features_hid=256, n_features_out=16):
        super(FeatureExtractor, self).__init__()

        self.n_features_in = n_features_in
        self.n_features_hid = n_features_hid
        self.n_features_out = n_features_out

        self.conv1 = torch.nn.Conv1d(n_features_in, n_features_hid, 1)
        self.conv2 = torch.nn.Conv1d(n_features_hid, n_features_out, 1)

    def forward(self, point_cloud):
        point_cloud = point_cloud.transpose(2, 1)
        point_cloud = F.relu(self.conv1(point_cloud))
        features = self.conv2(point_cloud)
        features = features.transpose(1, 2)
        return features
    
@timing
def grid_pooling(point_cloud, point_features_map, N=32):
    return GPooling.grid_pooling(point_cloud, point_features_map, N)

In [5]:
feature_extractor = FeatureExtractor()

with torch.no_grad():
    point_cloud_tensor = torch.tensor(point_cloud)[None, :, :].float()
    print(f'Point cloud shape: {point_cloud_tensor.shape}')
    features = feature_extractor(point_cloud_tensor)
    print(f'Feature maps shape: {features.shape}')
    grid = grid_pooling(point_cloud_tensor, features, N=32)
    print(f'Grid shape: {grid.shape}')

Point cloud shape: torch.Size([1, 75468, 3])
Feature maps shape: torch.Size([1, 75468, 16])
--- Function 'grid_pooling' took 0.75s to run ---
Grid shape: torch.Size([1, 32, 32, 32, 16])
